<a href="https://colab.research.google.com/github/cellatlas/human/blob/master/markers/lung/markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gget
!pip install -q git+https://github.com/sbooeshaghi/ec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import numpy as np
from ec.utils import write_markers

In [3]:
# Extract list of valid gene names in Ensembl release 96
!gget ref human -r "96" -ftp -w "gtf" -d

# Gunzip gtf
!gunzip /content/Homo_sapiens.GRCh38.96.gtf.gz

# Extract gene names
!tail -n +6 /content/Homo_sapiens.GRCh38.96.gtf   | cut -f 9 -d$'\t' | grep -v "transcript_id" | cut -f 6 -d" " | sed 's/"//g' | sed 's/;//'  | sort | uniq > genes.txt
genes_list = pd.read_csv('genes.txt', header = None)[0].values

Fri Mar 17 18:00:17 2023 INFO Fetching reference information for homo_sapiens from Ensembl release: 96.
http://ftp.ensembl.org/pub/release-96/gtf/homo_sapiens/Homo_sapiens.GRCh38.96.gtf.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.4M  100 42.4M    0     0   475k      0  0:01:31  0:01:31 --:--:--  479k


# Lung

In [18]:
species = "homo_sapiens"
organ = "lung"
reference = "GRCh38-Ensemble91"
paper_doi = "https://doi.org/10.1126/sciadv.aba1983"
table_link_1 = "https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s2.txt"
table_link_2 = "https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s3.txt"
table_link_3 = "https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s4.txt"
table_link_4 = "https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s5.txt"

table_links = [table_link_1, table_link_2, table_link_3, table_link_4]

# don't include in header
table_names = [
  "degs1.txt",
  "degs2.txt",
  "degs3.txt",
  "degs4.txt"
]

header = [
    {
      "species": species,
      "organ": organ,
      "reference": reference,
      "paper_doi": paper_doi,
      "table_link": table_link_1,
    },
    {
      "species": species,
      "organ": organ,
      "reference": reference,
      "paper_doi": paper_doi,
      "table_link": table_link_2,
    },
    {
      "species": species,
      "organ": organ,
      "reference": reference,
      "paper_doi": paper_doi,
      "table_link": table_link_3,
    },
    {
      "species": species,
      "organ": organ,
      "reference": reference,
      "paper_doi": paper_doi,
      "table_link": table_link_4,
    }
]

In [ ]:
counter = 1
fnames = []
for table_link in table_links:
  !wget --user-agent="Mozilla/4.9 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36" $table_link


In [15]:
df = pd.concat([pd.read_csv(name, sep = "\t") for name in table_names])
df.columns = ["celltype", "gene", "log1pc_FC", "pct_inGroup", "pct_outGroup", "wilcox_p", "wilcox_fdr", "wilcox_bonferroni"]


In [16]:
# Filter out genes not present in reference
bidx = df['gene'].isin(genes_list)
print(f'Filtered {np.sum(~bidx)} out of {len(bidx)} genes')
df = df[bidx]

Filtered 938 out of 20516 genes


In [17]:
df.head()

,celltype,gene,log1pc_FC,pct_inGroup,pct_outGroup,wilcox_p,wilcox_fdr,wilcox_bonferroni
1,Basal,MIR205HG,1.757514,0.969231,0.489669,8.086494e-39,2.838360e-36,2.838360e-36
2,Basal,KRT5,0.750194,0.861538,0.264463,3.598166e-36,6.314782e-34,1.262956e-33
3,Basal,EYA2,1.436151,0.984615,0.669421,1.463281e-33,1.712039e-31,5.136117e-31
4,Basal,CYP24A1,0.678724,0.892308,0.320248,4.873706e-32,4.276677e-30,1.710671e-29
5,Basal,KRT17,1.943518,0.953846,0.526860,2.228007e-31,1.564061e-29,7.820306e-29


In [29]:
min_mean = 100
max_pval = 1e-4
min_lfc = 0.8
max_gene_shares = 5
max_per_celltype = 20

# filter by criteria
dfc = df.query(f"wilcox_bonferroni <= {max_pval} & log1pc_FC >= {min_lfc}")

# mask out genes that are shared between max_gene_shares cell type
non_repeat_genes = dfc["gene"].value_counts()[dfc["gene"].value_counts() < max_gene_shares].index.values

m = dfc[dfc.gene.isin(non_repeat_genes)].sort_values('pct_inGroup', ascending = True)

# max number to sample is equal to the min number of genes across all celltype
n_sample = min(m["celltype"].value_counts().min(), max_per_celltype)

# sample n_sample genes
markers = m.groupby('celltype').tail(14) # Got 14 markers manually because some celltypes had very few markers
markers_dict = markers.groupby("celltype")["gene"].apply(lambda x: list(x)).to_dict()


In [32]:
markers.value_counts()

celltype     gene     log1pc_FC  pct_inGroup  pct_outGroup  wilcox_p      wilcox_fdr    wilcox_bonferroni
AT1          APLP2    0.846304   1.000000     0.930818      2.115587e-27  2.087992e-26  1.440714e-24         1
             B2M      1.110599   1.000000     0.987421      4.466875e-26  4.002555e-25  3.041942e-23         1
T Cytotoxic  AOAH     0.930803   1.000000     0.714004      1.483928e-17  6.121202e-17  2.448481e-15         1
T            RNF19A   0.933066   1.000000     0.928854      1.893792e-25  1.821647e-24  3.825459e-23         1
             RBPJ     0.867936   0.974026     0.915020      3.850615e-22  2.222355e-21  7.778242e-20         1
                                                                                                            ..
ILC A        SRGN     0.859028   1.000000     0.984314      6.485437e-19  7.097950e-18  1.277631e-16         1
             RIN3     0.971624   0.958904     0.866667      2.290200e-24  6.445277e-23  4.511694e-22         1
      

In [34]:
write_markers("markers.txt", markers_dict, header)

In [35]:
!cat markers.txt

# homo_sapiens	lung	GRCh38-Ensemble91	https://doi.org/10.1126/sciadv.aba1983	https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s2.txt
# homo_sapiens	lung	GRCh38-Ensemble91	https://doi.org/10.1126/sciadv.aba1983	https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s3.txt
# homo_sapiens	lung	GRCh38-Ensemble91	https://doi.org/10.1126/sciadv.aba1983	https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s4.txt
# homo_sapiens	lung	GRCh38-Ensemble91	https://doi.org/10.1126/sciadv.aba1983	https://www.science.org/doi/suppl/10.1126/sciadv.aba1983/suppl_file/aba1983_data_s5.txt
AT1	GPRC5A,EMP2,S100A10,LIMCH1,SFTA2,LMO7,MYL6,SPTBN1,MAGI1,CCSER1,B2M,APLP2,S100A6,TMSB4X
AT2	ABCA3,CMAHP,DLG2,ROS1,LRRK2,SNX25,ATP11A,TANC2,AKAP13,SFTPB,PTPRG,MACROD2,ANK3,PDE4D
Aberrant Basaloid	ITGB6,ITGA2,PPP1CB,PTPRE,RAB11FIP1,LDLRAD4,MRTFA,PTPRK,ZBTB20,ITGAV,PON2,CCSER1,LINC00511,TMSB4X
B	TRIO,PRKCE,PRKCB,EBF1,MGAT5,ARHGAP24,ADAM28

In [39]:
markers.groupby("celltype")["pct_inGroup"].mean().sort_values()

celltype
PNEC                   0.931548
Pericytes              0.942857
ILC B                  0.946154
Basal                  0.957265
Club                   0.970588
ILC A                  0.970646
Mast                   0.979437
T Regulatory           0.983903
AT2                    0.986486
Goblet                 0.986813
T                      0.988868
Myofibroblast          0.990816
SMC                    0.991228
B Plasma               0.992647
T Cytotoxic            0.993421
VE Venous              0.993789
VE Peribronchial       0.993842
VE Capillary B         0.994505
DC Langerhans          0.995714
VE Capillary A         0.996032
VE Arterial            0.997899
ncMonocyte             0.998120
Lymphatic              0.999022
Fibroblast             0.999035
cDC1                   1.000000
cDC2                   1.000000
cMonocyte              1.000000
AT1                    1.000000
Mesothelial            1.000000
Macrophage Alveolar    1.000000
Macrophage             1.000000